<a href="https://colab.research.google.com/github/vladijarosz/VJaroszewski-econ/blob/main/Tipos_de_cambio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns

In [31]:
!wget -O dolar_oficial.xlsx "https://github.com/vladijarosz/Mercado-de-Valores/raw/refs/heads/main/datos/dolar_oficial.xlsx"

import pandas as pd
df_oficial = pd.read_excel("dolar_oficial.xlsx")
df_oficial['Venta'] = (
    df_oficial['Venta']
    .astype(str)                     # 1) unificar tipo
    .str.replace('.', '', regex=False)   # 2) quitar miles
    .str.replace(',', '.', regex=False)  # 3) normalizar decimal
)

df_oficial['Venta'] = pd.to_numeric(df_oficial['Venta'], errors='coerce')  # 4) volver a número

df_oficial.sort_values('Fecha', inplace=True)
df_oficial

--2026-02-06 00:22:27--  https://github.com/vladijarosz/Mercado-de-Valores/raw/refs/heads/main/datos/dolar_oficial.xlsx
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/vladijarosz/Cuenta-inv/raw/refs/heads/main/datos/dolar_oficial.xlsx [following]
--2026-02-06 00:22:27--  https://github.com/vladijarosz/Cuenta-inv/raw/refs/heads/main/datos/dolar_oficial.xlsx
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/vladijarosz/Cuenta-inv/refs/heads/main/datos/dolar_oficial.xlsx [following]
--2026-02-06 00:22:27--  https://raw.githubusercontent.com/vladijarosz/Cuenta-inv/refs/heads/main/datos/dolar_oficial.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to r

,Fecha,Compra,Venta
5971,2002-04-09,"2,70",2.85
5970,2002-04-10,"2,70",2.84
5969,2002-04-11,"2,69",2.90
5968,2002-04-12,"2,83",3.05
5967,2002-04-15,"2,90",3.05
...,...,...,...
4,2026-01-28,"1413,25",1463.92
3,2026-01-29,"1414,63",1465.08
2,2026-01-30,"1414,11",1465.01
1,2026-01-31,"1414,11",1465.01


In [32]:
!wget -O MEP.csv "https://github.com/vladijarosz/Cuenta-inv/raw/refs/heads/main/datos/MEP.csv"

import pandas as pd
df_mep = pd.read_csv("MEP.csv")
df_mep['fecha'] = pd.to_datetime(df_mep['fecha'])
df_mep['mep'] = pd.to_numeric(df_mep['ultimo'], errors='coerce')
df_mep.sort_values('fecha', inplace=True)
df_mep['fecha']=pd.to_datetime(df_mep['fecha'])
df_mep.set_index('fecha', inplace=True)
mep=df_mep['ultimo']
mep=mep.loc["2023-01-01":"2024-12-31"]


--2026-02-06 00:22:28--  https://github.com/vladijarosz/Cuenta-inv/raw/refs/heads/main/datos/MEP.csv
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/vladijarosz/Cuenta-inv/refs/heads/main/datos/MEP.csv [following]
--2026-02-06 00:22:28--  https://raw.githubusercontent.com/vladijarosz/Cuenta-inv/refs/heads/main/datos/MEP.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149475 (146K) [text/plain]
Saving to: ‘MEP.csv’

MEP.csv             100%[===================>] 145.97K  --.-KB/s    in 0.02s   

2026-02-06 00:22:28 (6.26 MB/s) - ‘MEP.csv’ saved [149475/149475]



In [33]:
!wget -inflacion.xlsx "https://github.com/vladijarosz/Cuenta-inv/raw/refs/heads/main/datos/inflacion.xlsx"

df = pd.read_excel("inflacion.xlsx")
df = df.rename(columns={df.columns[0]: "fecha"})
df = df.rename(columns={df.columns[1]: "IPC"})
df.set_index("fecha", inplace=True)
df.index = pd.to_datetime(df.index)
inflacion_2024=df.loc["2023-01-01":"2024-12-31"]
inflacion_2024['IPC_indice'] = (1 + inflacion_2024['IPC'] / 100).cumprod() * 100

# Ordenamos en orden ascendente para calcular bien el acumulado
inflacion_2024 = inflacion_2024.sort_index(ascending=False)

# Calculamos el índice acumulado del IPC (asumiendo que IPC es inflación mensual en %)
inflacion_2024['factor'] = (1 + inflacion_2024['IPC'].shift(1)/100).cumprod()

# Si quieres el deflactor relativo al último mes (más reciente), normalizas
inflacion_2024['deflactor'] = 100
inflacion_2024['deflactor'] = inflacion_2024['deflactor'].iloc[-1] / inflacion_2024['factor']

inflacion_2024['deflactor'].iloc[0]=100


--2026-02-06 00:22:28--  https://github.com/vladijarosz/Cuenta-inv/raw/refs/heads/main/datos/inflacion.xlsx
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/vladijarosz/Cuenta-inv/refs/heads/main/datos/inflacion.xlsx [following]
--2026-02-06 00:22:28--  https://raw.githubusercontent.com/vladijarosz/Cuenta-inv/refs/heads/main/datos/inflacion.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12282 (12K) [application/octet-stream]
Saving to: ‘inflacion.xlsx.4’

inflacion.xlsx.4    100%[===================>]  11.99K  --.-KB/s    in 0s      

2026-02-06 00:22:28 (111 MB/s) - ‘inflacion.xlsx.4’ save

/tmp/ipython-input-2369097991.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipython-input-2369097991.py:21: FutureWarning:

ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps up

In [34]:
df_oficial.set_index('Fecha', inplace=True)
oficial=df_oficial['Venta']
oficial=oficial.loc["2023-01-01":"2024-12-31"]
divisas=pd.merge(mep, oficial, left_index=True, right_index=True)
divisas.columns=['ultimo','venta']
divisas=divisas.rename(columns={'ultimo':'mep',
                                'venta':'oficial'})
divisas.sort_index(ascending=False, inplace=True)
divisas


,mep,oficial
2024-12-30,1170.41,1060.28
2024-12-27,1170.56,1057.97
2024-12-26,1170.89,1055.82
2024-12-23,1162.30,1055.07
2024-12-20,1142.20,1054.27
...,...,...
2023-01-06,329.91,186.61
2023-01-05,332.67,186.25
2023-01-04,334.93,186.22
2023-01-03,330.70,185.99


In [57]:
divisas.loc['2023-12-07', 'mep']


np.float64(986.37)

In [58]:
divisas.loc['2023-12-13', 'oficial']

np.float64(832.64)

In [35]:

import plotly.graph_objects as go
import pandas as pd


fig = go.Figure()

# Línea dólar MEP
fig.add_trace(
    go.Scatter(
        x=divisas.index,
        y=divisas['mep'],
        mode='lines',
        name='Dólar MEP',
        line=dict(color='green', width=2)
    )
)

# Línea dólar MEP
fig.add_trace(
    go.Scatter(
        x=divisas.index,
        y=divisas['oficial'],
        mode='lines',
        name='Dólar Oficial',
        line=dict(color='blue', width=2)
    )
)


# Layout con selector temporal
fig.update_layout(
    title='Evolución del dólar MEP y oficial',
    xaxis_title='Fecha',
    yaxis_title='Precio (ARS)',
    xaxis=dict(
        rangeselector=dict(
            buttons=[
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=3, label="3m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="1a", step="year", stepmode="backward"),
                dict(step="all", label="Todo")
            ]
        ),
        rangeslider=dict(visible=True),
        type="date"
    ),
    hovermode='x unified',
    template='plotly_white'
)

fig.show()

In [36]:
# Create the 'mes' column as the first day of each month
divisas['mes'] = divisas.index.to_period('M').to_timestamp()
inflacion_2024['mes'] = inflacion_2024.index.to_period('M').to_timestamp()

# Hacer merge por mes
df = pd.merge(
    inflacion_2024[['mes', 'deflactor']], divisas[['mes','mep','oficial']], on='mes', how='left'
)
df['fecha']=divisas.index

df['mep_deflac']=df['mep']/(df['deflactor'])*100
df['oficial_deflac']=df['oficial']/(df['deflactor'])*100
df

,mes,deflactor,mep,oficial,fecha,mep_deflac,oficial_deflac
0,2024-12-01,100.000000,1170.41,1060.28,2024-12-30,1170.410000,1060.280000
1,2024-12-01,100.000000,1170.56,1057.97,2024-12-27,1170.560000,1057.970000
2,2024-12-01,100.000000,1170.89,1055.82,2024-12-26,1170.890000,1055.820000
3,2024-12-01,100.000000,1162.30,1055.07,2024-12-23,1162.300000,1055.070000
4,2024-12-01,100.000000,1142.20,1054.27,2024-12-20,1142.200000,1054.270000
...,...,...,...,...,...,...,...
538,2023-01-01,15.647363,329.91,186.61,2023-01-06,2108.406440,1192.597150
539,2023-01-01,15.647363,332.67,186.25,2023-01-05,2126.045196,1190.296443
540,2023-01-01,15.647363,334.93,186.22,2023-01-04,2140.488524,1190.104717
541,2023-01-01,15.647363,330.70,185.99,2023-01-03,2113.455215,1188.634821


In [37]:
import plotly.express as px

fig = px.line(
    df,
    x='fecha',
    y=['mep_deflac', 'oficial_deflac'],
    title='Dólar a precios constantes (deflactado)',
    labels={
        'fecha': 'Fecha',
        'value': 'Índice real (base 100)',
        'variable': 'Tipo de dólar'
    }
)

fig.update_layout(
    template='plotly_white',
    hovermode='x unified'
)

fig.update_traces(line=dict(width=3))

fig.show()
